In [ ]:
import serial 
import asyncio
import os 
import threading
import time
import queue
import re
from IPython.display import clear_output, display

In [ ]:
class serial_worker(threading.Thread):
    def __init__(self, port, baudrate):
        self.stop_event = threading.Event()
        self.port = port
        self.baudrate = baudrate
        self.to_device = queue.Queue()
        self.busy = False
        self._reader = None
        self._writer = None
        super().__init__()

    def connect(self):
        try:
            self._reader = serial.Serial(self.port, self.baudrate, timeout=1)
            print('connected')
            return True
        except Exception as e:
            print(e)
            return False

    def disconnect(self):
        self._reader.close()
        self._reader = None

    def is_busy(self):
        return self.busy

    def send(self, data):
        self.to_device.put(data)

    def is_connected(self):
        #check if the serial port is open and if the device is connected
        if self._reader is not None and self._reader.isOpen():
            return True
        print('serial port is not open')
        return False

    def temp_search(self, data):
        temp_regex_results = re.search('T:([^\s]+) \/([^\s]+) @:([^\s]+)', data.decode())
        if temp_regex_results:
            current_temp = temp_regex_results.group(1)
            target_temp = temp_regex_results.group(2)
            power = temp_regex_results.group(3)
            print(current_temp, target_temp, power)
    def position_search(self, data):
        position_regex_results = re.search('X:([^\s]+) Y:([^\s]+) Z:([^\s]+) E:([^\s]+)', data.decode())
        if position_regex_results:
            x = float(position_regex_results.group(1))
            y = float(position_regex_results.group(2))
            z = float(position_regex_results.group(3))
            e = float(position_regex_results.group(4))
            print(x, y, z, e)
    def reader(self):
        data = self._reader.readline()
        print('received: ', data.decode())
        if b'ok' in data:
            self.busy = False
            if self.to_device.empty():
                self.to_device.task_done()
        if b'echo:busy: processing' in data:
            # clear_output()
            # display('busy: processing')
            self.busy = True
        self.temp_search(data)
        self.position_search(data)
    def writer(self):
        if not self.busy:
            to_be_sent = self.to_device.get()
            to_be_sent = to_be_sent + '\r'
            print('sending: ', to_be_sent)
            self._reader.write(to_be_sent.encode())
            self._reader.flush()
    def run(self):
        print('thread started')
        if self.connect():
            while not self.stop_event.is_set():
                self.writer()
                self.reader()
        print('thread stopped')
        self.disconnect()
        self.stop_event.clear()
        self.to_device.queue.clear()


In [ ]:
ser = serial_worker('COM3', 115200)
ser.start()
ser.send('G1 X100 F10000')

In [ ]:
position= []
for i in range(0, 100, 10):
    position.append(i)
repetitions = 100
speed = 15000
while repetitions > 0:
    selected = repetitions % len(position)
    ser.send('G1 X{} F{}'.format(position[selected], speed))
    repetitions -= 1
    ser.to_device.join()
    print('done')
    

In [ ]:
# ser.stop_event.set()